# This colab
- Takes the Dataframe processed earlier
- embeddes 'product-description" into Chroma Vector DB
- Adds the following as metadata
- Adds the image ids s id

# Install Necessay stuff

In [1]:
!pip install langchain_community
!pip install chromadb
!pip install bitsandbytes

# Ref: https://python.langchain.com/docs/integrations/vectorstores/chroma/
!pip install -qU langchain-huggingface

!pip install -qU "langchain-chroma>=0.1.2"

In [2]:
import warnings
warnings.filterwarnings('ignore')



In [3]:
# mount storage
from google.colab import drive
drive.mount('/content/drive')
#%cd "/gdrive/MyDrive/Interview Kickstart/MLSwitchup/Capstone/ShopTalk"
%cd /content/drive/MyDrive/ik-ml/capstone/github/ShopTalk/
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ik-ml/capstone/github/ShopTalk
'Agent workflow sketch.pptx'  'BLIP-2 Fine-Tune.docx'   python_envs/
 assets/                       code/                    saved-data/


In [4]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from tqdm import tqdm
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import numpy as np # Import numpy


class VectorStore ():
  def __init__(self):
    self.preprocessed_data = None
    self.preprocessed_data_file = None
    self.content_col_name = None
    self.metadata_columns = None
    self.vector_store = None
    self.collection_name = None
    self.db_file_path = None
    self.embeddings = None
    self.retriever = None
    self.mdf = None


  # Takes the preprocessed dataframe saved as pickle file
  def loadPreprocessedData(self, pre_processed_data_file, content_col_name):
    self.preprocessed_data_file = pre_processed_data_file
    self.preprocessed_data = pd.read_pickle(self.preprocessed_data_file)
    self.preprocessed_data.dropna(subset=[content_col_name], inplace=True)
    return True

  def getData(self):
    return self.mdf

  def getPreprocessedData(self):
    return self.preprocessed_data

  # if the data file exists, DB will be created from the file. if not, new DB is created
  def createVectorStore(self, collection_name, db_file_path, embeddings):
    self.collection_name = collection_name
    self.db_file_path = db_file_path
    self.embeddings = embeddings

    self.vector_store = Chroma(collection_name=self.collection_name, embedding_function=self.embeddings,
                        persist_directory=self.db_file_path,  # Where to save data locally, remove if not necessary
                    )
    return True

  def getVectorStore(self):
    return self.vector_store

  def format_selected_columns(self, df, excluded_columns, existing_column_name):
    """
    Formats all columns except those specified in excluded_columns into "<column name> : <value>" text
    and appends to an existing column.

    Args:
      df: The input DataFrame.
      excluded_columns: A list of column names to exclude from formatting.
      existing_column_name: The name of the existing column to append the formatted text to.

    Returns:
      The DataFrame with the existing column modified.
    """

    # Get all columns except those in excluded_columns
    selected_columns = [col for col in df.columns if col not in excluded_columns]

    # Loop through rows and format selected columns, appending to existing column
    for index, row in df.iterrows():
      formatted_text = ""
      for col in selected_columns:
        formatted_text += f"{col} : {row[col]} "  # Format text

      # Ensure the existing value is a string before appending
      current_value = df.loc[index, existing_column_name]
      df.loc[index, existing_column_name] = str(current_value) + " " + formatted_text.strip()  # Append formatted text to existing column, ensuring string type
      #df.loc[index, existing_column_name] += " " + formatted_text.strip()  # Append formatted text to existing column

    return df



  #db = Chroma.from_documents(texts, embeddings)

  def add_data_to_vector_store(self, content_col_name):
    self.content_col_name = content_col_name
    #self.metadata_columns = metadata_cols_list

    total_count = self.preprocessed_data.shape[0]
    print('Total Records to add {}'.format(total_count))

    excluded_columns = ['product_description']  # List of columns to exclude
    existing_column_name = 'product_description'

    # convert the metadata columns
    #mdf = self.preprocessed_data[self.metadata_columns]
    self.mdf = self.format_selected_columns(self.preprocessed_data, excluded_columns, existing_column_name)
    count = 0
    skipped_count = 0
    for index, row in enumerate(tqdm(self.mdf.itertuples(), total=total_count)):
      try:
        count = count +1
        # Convert the namedtuple 'row' to a dictionary
        row_dict = row._asdict()
        text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
        texts = text_splitter.split_text(getattr(row, self.content_col_name))
        self.vector_store.add_texts(texts = texts, ids=row.Index)

      except Exception as e:
        skipped_count = skipped_count + 1
        print('--------------- Exception ------------------')
        print(e)
        print('row in error: index {}'.format(row.Index))
        print('row in error: content_col_name {}'.format(getattr(row, self.content_col_name)))
        print('---------------------------------------------')


    print('Skipped {} rows out of total rows {}'.format(skipped_count, count))
    return True


  def addDataToVectorStoreIteratively(self, content_col_name, metadata_cols_list):
      self.content_col_name = content_col_name
      self.metadata_columns = metadata_cols_list

      total_count = self.preprocessed_data.shape[0]
      print('Total Records to add {}'.format(total_count))

      # convert the metadata columns
      #mdf = self.preprocessed_data[self.metadata_columns]
      count = 0
      skipped_count = 0
      for index, row in enumerate(tqdm(self.preprocessed_data.itertuples(), total=total_count)):
        try:
          count = count +1
          # Convert the namedtuple 'row' to a dictionary
          row_dict = row._asdict()
          print('Processing row: {}'.format(count))
          mds=self.row_to_dict(row_dict, metadata_cols_list)
          idlist=[row_dict['Index']]
          text_to_add = row_dict[self.content_col_name]

          if ( text_to_add == '' or text_to_add == []):
            skipped_count = skipped_count + 1
            print(' -> Skipping row with index {} as the text to add is empty'.format(row_dict['Index']))
            continue

          text_list = [text_to_add]

          self.vector_store.add_texts(
              embedding_function=self.embeddings,
              collection_name=self.collection_name,
              ids=idlist, # Access 'Index' using dictionary key
              texts = text_list, # Access content column using dictionary key
              metadatas=mds  # Pass dictionary to row_to_dict
          )

        except Exception as e:
          print('--------------- Exception ------------------')
          print(e)
          print('row in error: index {}'.format(idlist))
          print('row in error: content_col_name {}'.format(text_list))
          print('row in error: metadata_cols_list {}'.format(mds))
          print('---------------------------------------------')

      print('Out of Total records {}, {} are skipped due to data issues'.format(count, skipped_count))
      return True

  def row_to_dict(self, row, column_names):
      """Converts a DataFrame row (now a dictionary) to a dictionary with specified columns and null value handling.
      """

      dict_list = []
      for column_name in column_names:
          value = row.get(column_name) # Use get() for dictionary access
          row_dict = {}
          # Check if value is a list or a NumPy array
          if isinstance(value, (list, np.ndarray)):
              # If it's a list or array, convert NaN values to empty strings within the list/array
              value = ["" if pd.isnull(x) else x for x in value]
          else:
              # If it's a single value, check for NaN and replace with an empty string
              if pd.isnull(value):
                  value = ''
          row_dict[column_name] = value

      dict_list.append(row_dict)
      return dict_list


  def getRetriever(self):
    if ( self.retriever is None):
      self.retriever = self.vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1, "fetch_k": 5})
    return self.retriever

  def addDataToVectorStoreBulk(self, content_col_name, metadata_cols_list):
    self.content_col_name = content_col_name
    self.metadata_columns = metadata_cols_list

    # convert the metadata columns
    mdf = self.preprocessed_data[self.metadata_columns]
    # Convert the product descriptions to strings before adding to ChromaDB
    # The tolist() method converts the Pandas Series into a list for ChromaDB
    self.vector_store.add_texts(
        embedding_function=self.embeddings,
        collection_name=self.collection_name,
        # if the df indexes are to be used as ids.if column content, then this requires change
        ids=self.preprocessed_data.index.tolist(),
        texts=self.preprocessed_data[content_col_name].astype(str).tolist(), # Convert to strings
        metadatas=mdf[metadata_cols_list].to_dict(orient="records") ,
    )
    return True



In [ ]:
# Test the class with dummay data
vs = VectorStore()
content_col_name = "product_description"
vs.loadPreprocessedData("./saved-data/abo-listings-english-fixed-product-type.pkl", content_col_name)
data = vs.getPreprocessedData()


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2516 entries, B08B85K5C2 to B07VF9MJ27
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   brand                2515 non-null   object
 1   bullet_point         2453 non-null   object
 2   color                1864 non-null   object
 3   fabric_type          281 non-null    object
 4   item_name            2516 non-null   object
 5   item_weight          1133 non-null   object
 6   model_name           287 non-null    object
 7   product_type         2516 non-null   object
 8   style                575 non-null    object
 9   main_image_id        2504 non-null   object
 10  other_image_id       2310 non-null   object
 11  country              2516 non-null   object
 12  marketplace          2516 non-null   object
 13  domain_name          2516 non-null   object
 14  material             1016 non-null   object
 15  item_keywords        2266 non-null   object
 

# Add by combaining all relevant columns to product_description

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vs.createVectorStore("st_products_all_in_one", "./saved-data/all_in_one_text_chroma_db", embeddings)


True

In [ ]:
vs.add_data_to_vector_store("product_description")

Total Records to add 2516


100%|██████████| 2516/2516 [03:09<00:00, 13.28it/s]

Skipped 0 rows out of total rows 2516


True

In [ ]:
retriever = vs.getRetriever()

In [ ]:
retriever.invoke("Give me all data that has color value set")
#retriever.invoke("Give me white color shoes list", filter={"source": "news"})

[Document(metadata={}, page_content="[] brand : [] bullet_point : [] color : beige fabric_type : nan item_name : UMI Essentials 2 x Eyelet Curtains item_weight : nan model_name : nan product_type : HOME style : nan main_image_id : 61Pgn0eErOL other_image_id : ['81n5YUIHE5L', '81DwZuLDzbL', '81oQUCqP66L', '71osl8VUjwL', '91ObAsy0qGL', '717v8yfpHwL', '71MZbRqR6-L'] country : DE marketplace : Amazon domain_name : amazon.de material : [] item_keywords : [] pattern : nan model_year : nan finish_type : nan")]

In [ ]:
retriever.invoke("Find items with brand as Whole Food Markets")

[Document(metadata={}, page_content="[] brand : Whole Foods Market bullet_point : ['100mg Green Tea Extract & Excellent Source of 4 B Vitamins.', 'Excellent source of Vitamins A & E.', 'Low Sodium.', 'Certified Kosher, Certified Vegan.', 'Brought to you by Whole Foods Market. When it comes to innovative flavors and products sourced from artisans and producers around the world, the Whole Foods Market brand has you covered. Amazing products, exceptional ingredients, no compromises.'] color : nan fabric_type : nan item_name : Whole Foods Market, Nutrient Enhanced Water Beverage, Cranberry + Antioxidants, 20 fl oz item_weight : 1.25 model_name : nan product_type : WATER style : nan main_image_id : 61uMrT6-vnL other_image_id : ['613ksNIe64L', '6146nquylaL', '61cW+tXQuAL'] country : US marketplace : Amazon domain_name : amazon.com material : nan item_keywords : ['Beverage SS Functional Beverages Single Serve Grocery & Gourmet Food Beverages Bottled Beverages, Water & Drink Mixes Water Flavor

# Add only product_description as text and rest of them as metadata columns

In [ ]:
# Test the class with dummay data
vs2 = VectorStore()
content_col_name = "product_description"
vs2.loadPreprocessedData("./saved-data/abo-listings-english-fixed-product-type.pkl", content_col_name)
data2 = vs2.getPreprocessedData()


In [ ]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2516 entries, B08B85K5C2 to B07VF9MJ27
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   brand                2515 non-null   object
 1   bullet_point         2453 non-null   object
 2   color                1864 non-null   object
 3   fabric_type          281 non-null    object
 4   item_name            2516 non-null   object
 5   item_weight          1133 non-null   object
 6   model_name           287 non-null    object
 7   product_type         2516 non-null   object
 8   style                575 non-null    object
 9   main_image_id        2504 non-null   object
 10  other_image_id       2310 non-null   object
 11  country              2516 non-null   object
 12  marketplace          2516 non-null   object
 13  domain_name          2516 non-null   object
 14  material             1016 non-null   object
 15  item_keywords        2266 non-null   object
 

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vs2.createVectorStore("product_data", "./saved-data/product_and_metadata_db", embeddings)

True

In [ ]:
metadata_cols_list = ['brand', 'color', 'fabric_type', 'item_name', 'item_weight', 'product_type','country', 'marketplace', 'item_keywords', 'model_year']
vs2.addDataToVectorStoreIteratively("product_description", metadata_cols_list )

Total Records to add 2516


  0%|          | 1/2516 [00:00<05:02,  8.31it/s]

Processing row: 1
Processing row: 2
Processing row: 3


  0%|          | 9/2516 [00:00<01:40, 24.89it/s]

Processing row: 4
 -> Skipping row with index B000V7RDCA as the text to add is empty
Processing row: 5
Processing row: 6
Processing row: 7
 -> Skipping row with index B07ZVWRWLC as the text to add is empty
Processing row: 8
 -> Skipping row with index B07G2L5KZC as the text to add is empty
Processing row: 9
Processing row: 10
 -> Skipping row with index B07QJG2F65 as the text to add is empty
Processing row: 11


...

100%|█████████▉| 2511/2516 [01:52<00:00, 23.32it/s]

Processing row: 2510
 -> Skipping row with index B00H4JMST6 as the text to add is empty
Processing row: 2511
Processing row: 2512
Processing row: 2513
Processing row: 2514


100%|██████████| 2516/2516 [01:52<00:00, 22.29it/s]

Processing row: 2515
Processing row: 2516
Out of Total records 2516, 580 are skipped due to data issues


True

In [ ]:
retriever2 = vs2.getRetriever()

In [ ]:
retriever2.invoke("Give me all data that has item_weight more than 10")
#retriever.invoke("Give me white color shoes list", filter={"source": "news"})

[Document(metadata={'model_year': ''}, page_content='BEST PRIDE FOR GREAT FAMILY PROTECTION - Made of high quality mesh polyester, our mesh laundry bags are sturdy, durable, plush and smooth. They can protect your clothes from damage caused by washing machine and avoid fabric knots There is no better pain when washing your expensive underwear, bras, and all other clothes with a great mesh wash bag. STAY ORGANIZED - Laundry bags in various sizes and colors can be used to wash or store different items of clothing. For example, a white laundry bag is used to clean light colored clothes, while black clothes are used for dark clothes. Different color zippers help you sort better clothes. QUIET & SECURE ZIPPER - Updated rust-proof zipper pulled into the protective band during washing ensures a quiet, knot-free and safe wash. A variety of laundry bag combinations are made to meet all your needs, to completely protect and store your various clothes! 5pcs Fine Net: 1XL Laundry Bag(60*60cm), 2L 